In [1]:
# import neccessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error, f1_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


Data Source: https://www.kaggle.com/c/intercampusai2019/data

In [2]:
# Get training data into pandas dataframe
train_data = pd.read_csv('./intercampusai2019/train.csv')

In [3]:
# Test data into pandas Dataframe
test_data = pd.read_csv('./intercampusai2019/test.csv')

In [4]:
train_data.columns

Index(['EmployeeNo', 'Division', 'Qualification', 'Gender',
       'Channel_of_Recruitment', 'Trainings_Attended', 'Year_of_birth',
       'Last_performance_score', 'Year_of_recruitment', 'Targets_met',
       'Previous_Award', 'Training_score_average', 'State_Of_Origin',
       'Foreign_schooled', 'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers',
       'Promoted_or_Not'],
      dtype='object')

In [5]:
train_data.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [6]:
test_data.columns

Index(['EmployeeNo', 'Division', 'Qualification', 'Gender',
       'Channel_of_Recruitment', 'Trainings_Attended', 'Year_of_birth',
       'Last_performance_score', 'Year_of_recruitment', 'Targets_met',
       'Previous_Award', 'Training_score_average', 'State_Of_Origin',
       'Foreign_schooled', 'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers'],
      dtype='object')

In [7]:
y = train_data.Promoted_or_Not
X = train_data.drop(['Promoted_or_Not', 'EmployeeNo'], axis=1)

In [8]:
# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                      random_state=0)                

In [9]:
# Select numerical columns
numerical_cols = [colname for colname in X_train.columns if X_train[colname].dtype in ['int64', 'float64']]

In [10]:
# Select categorical columns
cat_cols = [colname for colname in X_train.columns if X_train[colname].nunique() < 10 and X_train[colname].dtype == "object"]

In [11]:
cols = cat_cols + numerical_cols 
X_train1 = X_train[cols].copy()
X_valid1 = X_valid[cols].copy()
X_test = test_data[cols].copy()

In [12]:
X_train1.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average
14599,Regulatory and Legal services,"MSc, MBA and PhD",Male,Agency and others,Yes,Single,No,No,0,2,1980,12.5,2017,1,0,52
4,Information and Strategy,First Degree or HND,Male,Direct Internal process,Yes,Married,No,No,1,3,1990,7.5,2012,0,0,77
28190,Sourcing and Purchasing,"MSc, MBA and PhD",Male,Agency and others,Yes,Married,No,Yes,1,2,1985,10.0,2017,1,0,66
10683,Information Technology and Solution Support,First Degree or HND,Male,Agency and others,No,Married,No,No,1,2,1988,7.5,2014,0,0,69
10970,Sourcing and Purchasing,First Degree or HND,Female,Agency and others,Yes,Married,No,No,0,3,1992,2.5,2015,0,0,72


In [13]:
# Preprocessing for numerical data
numerical_transform = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
cat_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transform, numerical_cols),
        ('cat', cat_transform, cat_cols)
    ])

In [14]:
model1 = XGBClassifier(n_estimators=1000, learning_rate=0.01, max_depth=9, min_child_weight=9, 
                      subsample=0.9, colsample_bytree=0.9, reg_alpha=0.009)

In [15]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model1)
                             ])

In [16]:
# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='constant', verbose=0), ['Trainings_Attended', 'Year_of_birth....009, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.9, verbosity=1))])

In [17]:
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

In [18]:
# Evaluate the model
score = mean_absolute_error(y_valid, preds)
f1_score = f1_score(y_valid, preds)
print('MAE:', score)
print('f1_score:', f1_score)

MAE: 0.05598329635912828
f1_score: 0.546031746031746


In [19]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test).astype(int)

In [20]:
# Saving output to submission.csv
output = pd.DataFrame({'EmployeeNo': test_data.EmployeeNo,
                       'Promoted_or_Not': preds_test})
output.to_csv('submission.csv', index=False)